In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('adult.test.txt',header = None)
df_1 = pd.read_csv('adult.data.txt',header = None)
#frames = pd.concat([df,df_1])
frames = df.append(df_1)
frames

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K.
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K.
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K.
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K.


In [4]:
frames.columns=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']

# use only the continuous attributes, drop examples where there are missing values of the continuous attributes
X = frames[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']]
y = frames[['salary']]
y = y.astype(str)

LE = LabelEncoder()
y['binary_salary'] = LE.fit_transform(y['salary'])
#set(y['binary'])
y['binary_salary'].replace(1, -1 ,inplace=True)
y['binary_salary'].replace(3, 1 ,inplace=True)
y['binary_salary'].replace(0, -1 ,inplace=True)
y['binary_salary'].replace(2, 1 ,inplace=True)
del y['salary']

X = preprocessing.scale(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=101)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.125, random_state = 101)

In [6]:
def svm_fit(X,y,Lambda):
    
    m = 5   # constants chosen by experiment with small subsets of the dataset
    n = 5000
    a = np.random.rand(X.shape[1])
    b = np.random.rand(1)
    accuracy = []
    
    for epoch in range(50):
        step_size = m / (epoch+n)
        # set held out for the epoch
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=50)
        
        for step_count in range(300):
            if ((step_count+1) % 30 == 0):      # evaluate every 30 steps
                prediction = svm_predict((a,b), X_val)
                accuracy.append(accuracy_score(y_val, prediction))
                # TODO
            
            # pick random expamle
            randomize = random.choice(range(len(X_train)))
            #np.random.shuffle(randomize)
            y_train = np.array(y_train)
            X_rand=X_train[randomize]
            y_rand=y_train[randomize]
            
            # update our estimates using this gradient
            gamma = np.dot(a, X_rand) + b
            if (y_rand * gamma >= 1):   # predict a right sign
                a = a - step_size*Lambda*a
                b = b
            else:
                a = a - step_size*(Lambda*a - y_rand*X_rand)
                b = b - step_size*(-y_rand)
            
    return (a,b), accuracy
        
        

In [7]:
def svm_predict(param,X):
    a = param[0]
    b = param[1]
    prediction = np.zeros(len(X))
    for i,x in enumerate(X):
        if (np.dot(a,x)+b >= 0):
            prediction[i] = 1
        else:
            prediction[i] = -1
    return prediction

In [8]:
accuracy = []
best_accuracy = 0
#best_lambda = 1e-3
for Lambda in [1e-3, 1e-2, 1e-1, 1]:
    param, acc = svm_fit(X_train, y_train, Lambda)
    accuracy.append(acc)
    pred = svm_predict(param, X_val)
    score = accuracy_score(y_val, pred)
    if (score > best_accuracy):
        best_param = param
        best_lambda = Lambda
        best_accuracy = score

best_param, best_accuracy = svm_fit(X_train,y_train, best_lambda)
pred = svm_predict(best_param, X_test)
final_accuracy = accuracy_score(y_test, pred)

print("Best lambda: " + str(best_lambda))
print("Best Accuracy: "+ str(final_accuracy))
print("Best model: "+ str(best_param))

Best lambda: 0.01
Best Accuracy: 0.807778915046
Best model: (array([ 0.23955009,  0.01146652,  0.35683788,  1.06790807,  0.2355455 ,
        0.19036865]), array([-0.97110111]))
